## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json

import itertools
import math

from tqdm import tqdm

import pyDOE
import sys

from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render
import plotly.io as pio

import pickle
from matplotlib.pyplot import cm
import scienceplots

from utils import score_vals, param_to_list, list_to_param

# Mute warnings:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ax.plot.trace import optimization_trace_single_method

In [3]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [4]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10

## Start up client

In [5]:
Tag_client = AxClient()

[INFO 04-13 20:16:07] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [6]:
# Define bounds for each parameter type
int_bounds = {
    "z_sh": [6, 12], #NOTE - z_sh is actually half z_s (which must be even)
    "z_r2": [70, 86],
}

float_bounds = {
    "x_s": [-0.8, 0.8],
    "x_r2": [-1., 1.],
    # Cl is fixed as : 0.10-0.40
    # p1 & p2 offsets are fixed as : -1, 0, 1, 2
}

# Number of copies for each parameter
num_copies = 4

# Generate parameters dynamically
parameters = []

# Add integer parameters
for name, bounds in int_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (num_copies-1-i), upper - i],  # adjust bounds to deal with order
            "value_type": "int",
        })

# Add float parameters
for name, bounds in float_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (.1*(num_copies-1-i)), upper - .1*i],  # adjust bounds to deal with order
            "value_type": "float",
        })

# Add int / float parameters that make sure 

# Generate parameter constraints
parameter_constraints = []

# Ensure integer `z` values satisfy z_s - z_s_1 >= 1
for name in int_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 1")

# Ensure float `x` values satisfy x_s - x_s_1 >= 0.1
for name in float_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 0.1")

# Create the experiment
Tag_client.create_experiment(
    name="gear_experiment",
    parameters=parameters,
    objectives={"Cross_term_gear_ratio_product": ObjectiveProperties(minimize=False)},
    parameter_constraints=parameter_constraints,  # Add constraints here
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

# Print constraints for verification
# print("Parameter Constraints:")
# for constraint in parameter_constraints:
#     print(constraint)

[INFO 04-13 20:16:07] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[9, 12]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[8, 11]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[7, 10]), RangeParameter(name='z_sh_3', parameter_type=INT, range=[6, 9]), RangeParameter(name='z_r2', parameter_type=INT, range=[73, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[72, 85]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[71, 84]), RangeParameter(name='z_r2_3', parameter_type=INT, range=[70, 83]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[-0.5, 0.8]), RangeParameter(name='x_s_1', parameter_type=FLOAT, range=[-0.6000000000000001, 0.7000000000000001]), RangeParameter(name='x_s_2', parameter_type=FLOAT, range=[-0.7000000000000001, 0.6000000000000001]), RangeParameter(name='x_s_3', parameter_type=FLOAT, range=[-0.8, 0.5]), RangeParameter(name='x_r2', par

In [7]:
max_cross_terms = 4**(len(int_bounds) + len(float_bounds)+1) # plus Cl.
print(f'max_cross_terms: {max_cross_terms}')

max_cross_terms: 1024


In [8]:
# -1.0*z_p2 + 1.0*z_p2_1 <= -1.0
# -1.0*x_p2_1 + 1.0*x_p2 <= -0.01

In [9]:
parameters

[{'name': 'z_sh', 'type': 'range', 'bounds': [9, 12], 'value_type': 'int'},
 {'name': 'z_sh_1', 'type': 'range', 'bounds': [8, 11], 'value_type': 'int'},
 {'name': 'z_sh_2', 'type': 'range', 'bounds': [7, 10], 'value_type': 'int'},
 {'name': 'z_sh_3', 'type': 'range', 'bounds': [6, 9], 'value_type': 'int'},
 {'name': 'z_r2', 'type': 'range', 'bounds': [73, 86], 'value_type': 'int'},
 {'name': 'z_r2_1', 'type': 'range', 'bounds': [72, 85], 'value_type': 'int'},
 {'name': 'z_r2_2', 'type': 'range', 'bounds': [71, 84], 'value_type': 'int'},
 {'name': 'z_r2_3', 'type': 'range', 'bounds': [70, 83], 'value_type': 'int'},
 {'name': 'x_s',
  'type': 'range',
  'bounds': [-0.5, 0.8],
  'value_type': 'float'},
 {'name': 'x_s_1',
  'type': 'range',
  'bounds': [-0.6000000000000001, 0.7000000000000001],
  'value_type': 'float'},
 {'name': 'x_s_2',
  'type': 'range',
  'bounds': [-0.7000000000000001, 0.6000000000000001],
  'value_type': 'float'},
 {'name': 'x_s_3',
  'type': 'range',
  'bounds': [-

In [10]:
parameter_constraints

['z_sh - z_sh_1 >= 1',
 'z_sh_1 - z_sh_2 >= 1',
 'z_sh_2 - z_sh_3 >= 1',
 'z_r2 - z_r2_1 >= 1',
 'z_r2_1 - z_r2_2 >= 1',
 'z_r2_2 - z_r2_3 >= 1',
 'x_s - x_s_1 >= 0.1',
 'x_s_1 - x_s_2 >= 0.1',
 'x_s_2 - x_s_3 >= 0.1',
 'x_r2 - x_r2_1 >= 0.1',
 'x_r2_1 - x_r2_2 >= 0.1',
 'x_r2_2 - x_r2_3 >= 0.1']

### Run tests to solve for best possible score    

In [11]:
success = 0
fail_count = 0
while not success:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        success = 1
    except:
        fail_count += 1

print(f"Failed to get next trial {fail_count} times")

[WARNING 04-13 20:16:19] ax.modelbridge.transforms.int_to_float: Unable to round {'z_sh': 12.480590450328608, 'z_sh_1': 10.655292732211166, 'z_sh_2': 8.008086615198366, 'z_sh_3': 6.274857384587842, 'z_r2': 85.89453441212106, 'z_r2_1': 79.56672446963469, 'z_r2_2': 76.29099773849514, 'z_r2_3': 74.15134328105428, 'x_s': 0.56300787338987, 'x_s_1': 0.37237706007435933, 'x_s_2': -0.18275554832071061, 'x_s_3': -0.4775549834594131, 'x_r2': -0.048552756570279576, 'x_r2_1': -0.3450812319293618, 'x_r2_2': -0.8229293103329838, 'x_r2_3': -0.9377879871055483}to meet parameter constraints of SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[9, 12]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[8, 11]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[7, 10]), RangeParameter(name='z_sh_3', parameter_type=INT, range=[6, 9]), RangeParameter(name='z_r2', parameter_type=INT, range=[73, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[72, 85]),

Failed to get next trial 37 times


In [12]:
parameterization

{'z_sh': 12,
 'z_sh_1': 11,
 'z_sh_2': 8,
 'z_sh_3': 6,
 'z_r2': 86,
 'z_r2_1': 80,
 'z_r2_2': 76,
 'z_r2_3': 74,
 'x_s': 0.56300787338987,
 'x_s_1': 0.37237706007435933,
 'x_s_2': -0.18275554832071061,
 'x_s_3': -0.4775549834594131,
 'x_r2': -0.048552756570279576,
 'x_r2_1': -0.3450812319293618,
 'x_r2_2': -0.8229293103329838,
 'x_r2_3': -0.9377879871055483}

In [13]:
print(len(parameterization))
for name, value in parameterization.items():
    print(f"{name}: {value}")

16
z_sh: 12
z_sh_1: 11
z_sh_2: 8
z_sh_3: 6
z_r2: 86
z_r2_1: 80
z_r2_2: 76
z_r2_3: 74
x_s: 0.56300787338987
x_s_1: 0.37237706007435933
x_s_2: -0.18275554832071061
x_s_3: -0.4775549834594131
x_r2: -0.048552756570279576
x_r2_1: -0.3450812319293618
x_r2_2: -0.8229293103329838
x_r2_3: -0.9377879871055483


In [14]:
in_vals, offsets = param_to_list(parameterization, add_cl = True, add_offset = True)
Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(in_vals, offsets=offsets, add_gear = False))

[INFO 04-13 20:16:19] ax.service.ax_client: Completed trial 0 with data: {'Cross_term_gear_ratio_product': (256612.106763, None)}.


In [15]:
list = param_to_list(parameterization, add_cl = True)
print(list)

[array([12, 11,  8,  6]), array([86, 80, 76, 74]), array([ 0.56300787,  0.37237706, -0.18275555, -0.47755498]), array([-0.04855276, -0.34508123, -0.82292931, -0.93778799]), array([0.    , 0.0001, 0.0002, 0.0003])]


NOTE - run the following through 'Exhausted' errors.

In [16]:
init_count = 0
init_trials = 32 # hand set based on output at initialization
fail_count = 0
while init_count < init_trials:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        # Local evaluation here can be replaced with deployment to external system.
        in_vals, offsets = param_to_list(parameterization, add_cl = True, add_offset = True)
        Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(in_vals, offsets, add_gear = False))
    except:
        # print('Not this time')
        fail_count += 1
    init_count = trial_index
print(f'Failed {fail_count} times while using Sobol')

[WARNING 04-13 20:16:23] ax.modelbridge.transforms.int_to_float: Unable to round {'z_sh': 11.632593642364983, 'z_sh_1': 9.320189779022336, 'z_sh_2': 7.505107400301545, 'z_sh_3': 6.053098224134743, 'z_r2': 86.08869268510357, 'z_r2_1': 85.07021901180565, 'z_r2_2': 80.59832374697744, 'z_r2_3': 75.80717823908898, 'x_s': 0.2846376648172736, 'x_s_1': -0.41984260994941003, 'x_s_2': -0.686306430026889, 'x_s_3': -0.7877787010744215, 'x_r2': 0.6007116623222828, 'x_r2_1': 0.34219740321859726, 'x_r2_2': -0.13060049917548888, 'x_r2_3': -0.5824385141022503}to meet parameter constraints of SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[9, 12]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[8, 11]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[7, 10]), RangeParameter(name='z_sh_3', parameter_type=INT, range=[6, 9]), RangeParameter(name='z_r2', parameter_type=INT, range=[73, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[72, 85]), R

Failed 1249 times while using Sobol


In [17]:
Tag_client.generation_strategy.trials_as_df

,trial_index,arm_name,trial_status,generation_method,generation_node,Cross_term_gear_ratio_product,z_sh,z_sh_1,z_sh_2,z_sh_3,...,z_r2_2,z_r2_3,x_s,x_s_1,x_s_2,x_s_3,x_r2,x_r2_1,x_r2_2,x_r2_3
0,0,0_0,COMPLETED,Sobol,GenerationStep_0,256612.106763,12,11,8,6,...,76,74,0.563008,0.372377,-0.182756,-0.477555,-0.048553,-0.345081,-0.822929,-0.937788
1,1,1_0,COMPLETED,Sobol,GenerationStep_0,224654.934030,12,9,8,6,...,81,76,0.284638,-0.419843,-0.686306,-0.787779,0.600712,0.342197,-0.130600,-0.582439
2,2,2_0,COMPLETED,Sobol,GenerationStep_0,338360.937979,12,10,8,6,...,75,73,0.424231,0.119124,-0.082938,-0.347384,0.866357,0.374701,-0.195112,-0.754204
3,3,3_0,COMPLETED,Sobol,GenerationStep_0,298654.213232,12,11,8,7,...,77,71,0.617163,0.406270,-0.013384,-0.166312,0.196356,-0.015821,-0.604453,-0.808103
4,4,4_0,COMPLETED,Sobol,GenerationStep_0,359218.222726,12,11,7,6,...,78,73,0.775839,0.423635,-0.023326,-0.487352,0.743236,0.559427,0.382119,0.160438
5,5,5_0,COMPLETED,Sobol,GenerationStep_0,352604.014049,12,8,7,6,...,72,70,0.667489,0.278584,-0.385959,-0.636491,0.803289,0.348172,-0.292496,-0.394222
6,6,6_0,COMPLETED,Sobol,GenerationStep_0,350634.106683,12,9,7,6,...,78,70,0.562163,0.309692,-0.591653,-0.744131,0.479285,0.302865,-0.298044,-0.701835
7,7,7_0,COMPLETED,Sobol,GenerationStep_0,287423.486864,12,10,9,6,...,78,74,0.616879,0.279578,0.010203,-0.623644,0.961657,0.765130,0.510827,-0.827471
8,8,8_0,COMPLETED,Sobol,GenerationStep_0,283888.044470,11,10,9,7,...,78,70,0.775806,0.607814,-0.466956,-0.573432,0.506488,0.146868,-0.105040,-0.992409
9,9,9_0,COMPLETED,Sobol,GenerationStep_0,286173.168665,12,10,9,6,...,79,71,0.561889,-0.309653,-0.474020,-0.731762,0.460270,-0.040402,-0.414684,-0.962668


In [18]:
render(Tag_client.get_optimization_trace())

In [19]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    in_vals, offsets = param_to_list(parameterization, add_cl = True, add_offset = True)
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(in_vals, offsets, add_gear = False))

[INFO 04-13 20:23:51] ax.service.ax_client: Generated new trial 33 with parameters {'z_sh': 11, 'z_sh_1': 10, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 79, 'z_r2_1': 78, 'z_r2_2': 77, 'z_r2_3': 76, 'x_s': 0.8, 'x_s_1': -0.6, 'x_s_2': -0.7, 'x_s_3': -0.8, 'x_r2': 1.0, 'x_r2_1': -0.000102, 'x_r2_2': -0.100102, 'x_r2_3': -0.200102} using model BoTorch.
[INFO 04-13 20:23:51] ax.service.ax_client: Completed trial 33 with data: {'Cross_term_gear_ratio_product': (382538.746032, None)}.
[INFO 04-13 20:24:03] ax.service.ax_client: Generated new trial 34 with parameters {'z_sh': 9, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 78, 'z_r2_1': 77, 'z_r2_2': 76, 'z_r2_3': 75, 'x_s': 0.8, 'x_s_1': 0.507385, 'x_s_2': -0.253009, 'x_s_3': -0.8, 'x_r2': 0.283604, 'x_r2_1': 0.183604, 'x_r2_2': 0.083604, 'x_r2_3': -0.180075} using model BoTorch.
[INFO 04-13 20:24:03] ax.service.ax_client: Completed trial 34 with data: {'Cross_term_gear_ratio_product': (425541.564301, None)}.
[INFO 04-13 20:24:14] ax.service.ax_cl

In [20]:
best_parameters, values = Tag_client.get_best_parameters()
for name, value in best_parameters.items():
    print(f"{name}: {value}")

z_sh: 9
z_sh_1: 8
z_sh_2: 7
z_sh_3: 6
z_r2: 79
z_r2_1: 78
z_r2_2: 77
z_r2_3: 76
x_s: -0.011382934153691323
x_s_1: -0.5881379053729652
x_s_2: -0.6881379053730234
x_s_3: -0.7999999999999492
x_r2: 0.3530162719742339
x_r2_1: 0.2530162719741978
x_r2_2: 0.15301627197411177
x_r2_3: 0.05301627197396817


In [21]:
in_vals, offsets = param_to_list(parameterization, add_cl = True, add_offset = True)
print(f'\'Best\' score: {score_vals(in_vals, offsets, add_gear = False)}')

'Best' score: 435618.10733300564


In [22]:
param_to_list(best_parameters, add_cl = True)

[array([9, 8, 7, 6]),
 array([79, 78, 77, 76]),
 array([-0.01138293, -0.58813791, -0.68813791, -0.8       ]),
 array([0.35301627, 0.25301627, 0.15301627, 0.05301627]),
 array([0.    , 0.0001, 0.0002, 0.0003])]

In [23]:
render(Tag_client.get_optimization_trace())

In [24]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    in_vals, offsets = param_to_list(parameterization, add_cl = True, add_offset = True)
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(in_vals, offsets, add_gear = False))

[INFO 04-13 20:45:59] ax.service.ax_client: Generated new trial 133 with parameters {'z_sh': 9, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 78, 'z_r2_1': 77, 'z_r2_2': 76, 'z_r2_3': 75, 'x_s': -0.074104, 'x_s_1': -0.174104, 'x_s_2': -0.7, 'x_s_3': -0.8, 'x_r2': 0.585969, 'x_r2_1': 0.485969, 'x_r2_2': 0.385969, 'x_r2_3': 0.285969} using model BoTorch.
[INFO 04-13 20:45:59] ax.service.ax_client: Completed trial 133 with data: {'Cross_term_gear_ratio_product': (435998.971375, None)}.
[INFO 04-13 20:46:12] ax.service.ax_client: Generated new trial 134 with parameters {'z_sh': 9, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 78, 'z_r2_1': 77, 'z_r2_2': 76, 'z_r2_3': 72, 'x_s': 0.8, 'x_s_1': 0.7, 'x_s_2': -0.7, 'x_s_3': -0.8, 'x_r2': 1.0, 'x_r2_1': 0.9, 'x_r2_2': 0.8, 'x_r2_3': 0.7} using model BoTorch.
[INFO 04-13 20:46:12] ax.service.ax_client: Completed trial 134 with data: {'Cross_term_gear_ratio_product': (407803.897783, None)}.
[INFO 04-13 20:46:26] ax.service.ax_client: Generated 

In [25]:
render(Tag_client.get_optimization_trace())

In [26]:
for i in range(800):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    in_vals, offsets = param_to_list(parameterization, add_cl = True, add_offset = True)
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(in_vals, offsets, add_gear = False))

[INFO 04-13 21:12:14] ax.service.ax_client: Generated new trial 233 with parameters {'z_sh': 9, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 81, 'z_r2_1': 78, 'z_r2_2': 77, 'z_r2_3': 72, 'x_s': -0.014185, 'x_s_1': -0.114185, 'x_s_2': -0.214185, 'x_s_3': -0.8, 'x_r2': 0.337824, 'x_r2_1': 0.237824, 'x_r2_2': 0.137824, 'x_r2_3': 0.037824} using model BoTorch.
[INFO 04-13 21:12:14] ax.service.ax_client: Completed trial 233 with data: {'Cross_term_gear_ratio_product': (437697.388587, None)}.
[INFO 04-13 21:12:33] ax.service.ax_client: Generated new trial 234 with parameters {'z_sh': 9, 'z_sh_1': 8, 'z_sh_2': 7, 'z_sh_3': 6, 'z_r2': 81, 'z_r2_1': 78, 'z_r2_2': 77, 'z_r2_3': 76, 'x_s': -0.026895, 'x_s_1': -0.126895, 'x_s_2': -0.226895, 'x_s_3': -0.8, 'x_r2': 0.279631, 'x_r2_1': 0.179631, 'x_r2_2': 0.079631, 'x_r2_3': -0.020369} using model BoTorch.
[INFO 04-13 21:12:33] ax.service.ax_client: Completed trial 234 with data: {'Cross_term_gear_ratio_product': (454273.505468, None)}.
[INFO 04-13

In [27]:
render(Tag_client.get_optimization_trace())

In [28]:
best_parameters, values = Tag_client.get_best_parameters()
for name, value in best_parameters.items():
    print(f"{name}: {value}")

[INFO 04-14 07:59:04] ax.modelbridge.base: Leaving out out-of-design observations for arms: 1022_0, 574_0, 595_0, 618_0, 808_0, 986_0, 743_0, 586_0, 596_0, 825_0, 959_0, 987_0, 908_0, 962_0, 578_0, 1002_0, 614_0, 606_0, 835_0, 719_0, 768_0, 632_0, 922_0, 902_0, 934_0, 973_0, 1003_0, 714_0, 285_0, 646_0, 712_0, 580_0, 931_0, 710_0, 830_0, 793_0, 602_0, 769_0, 728_0, 594_0, 800_0, 621_0, 645_0, 1017_0, 644_0, 1027_0, 536_0, 626_0, 804_0, 623_0, 936_0, 601_0, 742_0, 556_0, 568_0, 704_0, 617_0, 989_0, 494_0, 537_0, 985_0, 562_0, 294_0, 559_0, 468_0, 829_0, 968_0, 600_0, 553_0, 637_0, 560_0, 598_0, 597_0, 815_0, 956_0, 965_0, 982_0, 673_0, 703_0, 1024_0, 636_0, 1029_0, 1011_0, 583_0, 816_0, 802_0, 624_0, 778_0, 870_0, 914_0, 709_0, 786_0, 599_0, 737_0, 805_0, 961_0, 834_0, 565_0, 838_0, 952_0, 976_0, 803_0, 827_0, 571_0, 579_0, 974_0, 629_0, 947_0, 995_0, 764_0, 951_0, 575_0, 910_0, 975_0, 963_0, 771_0, 971_0, 953_0, 642_0, 775_0, 633_0, 798_0, 519_0, 634_0, 727_0, 607_0, 635_0, 665_0, 708_

z_sh: 9
z_sh_1: 8
z_sh_2: 7
z_sh_3: 6
z_r2: 81
z_r2_1: 80
z_r2_2: 78
z_r2_3: 77
x_s: 0.0656521349063731
x_s_1: -0.03434786509783692
x_s_2: -0.13434786509838154
x_s_3: -0.7999999999985298
x_r2: 0.26201091373943153
x_r2_1: 0.16201091373929835
x_r2_2: 0.06201091373910563
x_r2_3: -0.037989086267733035


In [29]:
print(score_vals(param_to_list(best_parameters, add_cl = True), add_gear = False))

25600.4656272263


In [30]:
param_to_list(best_parameters, add_cl = True)

[array([9, 8, 7, 6]),
 array([81, 80, 78, 77]),
 array([ 0.06565213, -0.03434787, -0.13434787, -0.8       ]),
 array([ 0.26201091,  0.16201091,  0.06201091, -0.03798909]),
 array([0.    , 0.0001, 0.0002, 0.0003])]